### Índice
1. Sobre o conjunto de dados
2. Pré-processamento de texto
3. Usar TFIDF para melhorar o vetor de contagem
4. Conclusão

### 1. Sobre o dataset 

CEPs

Há um total de xxx CEPs no conjunto de dados..


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

#import nltk
#nltk.download('rslp')

# Para tratar as palavras dos textos
import unicodedata # Conjunto de caracteres de uniformidade unicode
import re # Regular Expression

# Para possibilitar processamento paralelo
from multiprocessing import  Pool

pd.set_option('display.max_colwidth', None)

In [2]:
arquivo = 'ect_amostra_minas_gerais.csv'
sep = ';'
# ler o dataset
df = pd.read_csv(arquivo, sep=sep)
#df.select_dtypes(exclude=[np.number]).columns
#df.shape
df.head()

,log_nu,ufe_sg,loc_nu,bai_nu_ini,bai_nu_fim,log_no,log_complemento,cep,int_cep,tlo_tx,log_sta_tlo,log_no_abrev,bai_no,bai_no_abrev,mun_nu
0,1008337,MG,4047,57341,0,Ouro Fino,NaN,38042274,38042274,Rua,S,R Ouro Fino,Damha Residencial Uberaba I,D Res Uberaba I,3170107
1,1008338,MG,4047,57341,0,Mariana,NaN,38042276,38042276,Rua,S,R Mariana,Damha Residencial Uberaba I,D Res Uberaba I,3170107
2,1008339,MG,4047,57341,0,Lagoa Formosa,NaN,38042278,38042278,Rua,S,R Lga Formosa,Damha Residencial Uberaba I,D Res Uberaba I,3170107
3,1008340,MG,4047,57341,0,A,NaN,38042280,38042280,Rua,S,R A,Damha Residencial Uberaba I,D Res Uberaba I,3170107
4,1008341,MG,4047,57341,0,Um,NaN,38042282,38042282,Avenida,S,Av Um,Damha Residencial Uberaba I,D Res Uberaba I,3170107


In [3]:
df.shape

(111880, 15)

https://pt.stackoverflow.com/questions/375592/como-cortar-texto-de-string-em-python-delimitando-a-substring-inicial-para-a-po  
def split_text(obj, substring = None, start = 0, qtd = None):
    qtd = len(obj) if qtd is None else qtd

    if substring:
        inicio = obj.find(substring)
        return obj[inicio:inicio+qtd]
    elif not substring:
        return obj[start:start+qtd]

In [4]:
# Coluna estratifica conté os 3 primeiros dígitos do CEP. Junto com ufe_sg estratifica forma o par
# para a estratificação proporcional dos dados.
df['estratifica'] = (df['cep']/100000).astype(int)

features_all = ['log_nu', 'ufe_sg', 'loc_nu', 'bai_nu_ini', 'bai_nu_fim', 'log_no', 
            'log_complemento', 'cep', 'tlo_tx', 'log_sta_tlo', 
            'log_no_abrev', 'bai_no', 'bai_no_abrev', 'mun_nu', 'estratifica']

features_num = ['log_nu','loc_nu','bai_nu_ini','bai_nu_fim','mun_nu']
features_txt = ['ufe_sg', 'log_no', 'log_complemento', 'tlo_tx', 'log_sta_tlo', 
                'log_no_abrev', 'bai_no', 'bai_no_abrev']

estratifica = ['ufe_sg','estratifica'] 

In [5]:
X_train, df_end, y_train, y_test = train_test_split(df[features_all], 
                                                    df[estratifica], 
                                                    test_size=0.13, 
                                                    random_state=42)

# O df_cep é criado para possibilitar a criação dos diferentes label com os dígitos do CEP.
df_cep = pd.DataFrame()
df_cep['cep'] = df_end['cep']

# Mantém o DataFrame df[features_all] com o mesmo número de registros depois da estratificação.
df = df_end

del X_train, y_train, y_test

In [6]:
df_cep.shape

(14545, 1)

In [7]:
def load_stoplist():
    stoplist = open("stopwords.txt", "r")
    stoplist = stoplist.read()
    stoplist = set(stoplist.splitlines())
    return stoplist

stopwords = load_stoplist()

In [8]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb
def parallelize_dataframe(df, func, n_cores=8):
    """
    Function that parallelizes any function applied to a dataframe
    Input:
    df      - Dataframe
    func    - Function to be aplied to portions of Dataframe
    n_cores - Number of CPU cores to be used on the application of func
    
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [9]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb

# Processo que cria a feature end_txt a partir dos campos que compõem um endereço
def preprocessing_df_txt(df):
    return df[features_txt].apply(lambda x: " ".join(x.astype(str)), axis=1)

In [10]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb

#Paralelização do processo que cria a nova feature end_txt
df_end['end_txt'] = parallelize_dataframe(df_end[features_txt], preprocessing_df_txt)
df_end['end_txt'] = [re.sub(r'nan', ' ', x) for x in df_end['end_txt'].str.lower()]
df_end.shape

(14545, 16)

'''
Aplica o conjunto unicode NFKD Compatibility Decomposition  
https://unicode.org/reports/tr15/   
Utilização: irá retirar todas as acentuações e deixará nas letras origianais  
Será útil quando os endereços estiverem mal escritos   
'''

def norma(old):
    new = ''.join(ch for ch in unicodedata.normalize('NFKD', 
                    str(old)) if not unicodedata.combining(ch))
    return new

Aplica a normalização sobre a feature end_txt  

df_end.end_txt = norma(df_end.end_txt)
df_end.head(1)

In [11]:
df_end['cep'] = df_cep.cep

# Depois de usado o DataFrame df_cep será apagado para liberar de RAM
del df_cep

df_end.shape

(14545, 16)

- A variável target **`label`** receberá **`4`** dígitos do CEP
- São acrescidos ao conjunto de `features` os `cep_2d` e `cep_3d`

In [12]:
# Será criada uma feature cep_2d 
df_end['cep_2d'] = [int(x/1000000) for x in df_end.cep]

# Será criada uma feature cep_3d
df_end['cep_3d'] = [int(x/100000) for x in df_end.cep]

# label CEP com 4 DÍGITOS
df_end['label'] = [int(x/10000) for x in df_end.cep]

df_end = df_end.drop(columns=['cep'])

# Mantém o DataFrame df[features_all] com o mesmo número de registros depois da estratificação.
df = df_end

df_end.label.value_counts()
#df_end.shape

3940    549
3570    390
3550    322
3840    322
3755    313
       ... 
3012      2
3473      2
3631      1
3305      1
3225      1
Name: label, Length: 269, dtype: int64

Quantidade de classes em `label` é `274`

In [13]:
df_end.head(1)

,log_nu,ufe_sg,loc_nu,bai_nu_ini,bai_nu_fim,log_no,log_complemento,tlo_tx,log_sta_tlo,log_no_abrev,bai_no,bai_no_abrev,mun_nu,estratifica,end_txt,cep_2d,cep_3d,label
24505,1046029,MG,3547,4936,0,Professora Maria Hipólita Lemos,NaN,Rua,S,R Prfa Maria H Lemos,Vila São José,Vl S José,3147907,379,mg professora maria hipólita lemos rua s r prfa maria h lemos vila são josé vl s josé,37,379,3790


### 4. TFIDF

O TFIDF também pode ser facilmente implementado em Python usando o Sklearn.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# inicializar TFIDF
vec = TfidfVectorizer(max_features=4000, stop_words=stopwords)
# criar TFIDF
tfidf = vec.fit_transform(df_end.end_txt)
#tfidf_df = vec.fit_transform(df_end.end_txt)

tfidf.shape
#tfidf_df.shape

(14545, 5000)

In [15]:
from scipy.sparse import csr_matrix #para transformar DataFrame em Sparse DataFrame otimizado

sp_vec = csr_matrix(tfidf)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(sp_vec)

# amostra
#tfidf_df.iloc[:20, 3000:3050]
tfidf_df.shape

(14545, 5000)

São acrescentadas as `features` que compõem o endereço + **`cep_2d`** e **`cep_3d`**

In [16]:
# São acrescentadas as features que compõem o endereço + cep_2d
tfidf_df = pd.concat([df[['log_nu','loc_nu','bai_nu_ini','bai_nu_fim','mun_nu',
                          'ufe_sg','estratifica',
                          'cep_2d','cep_3d','label']], tfidf_df], axis=1, join='inner')
tfidf_df.shape

(1925, 5010)

In [17]:
X_train, tfidf_df, y_train, y_test = train_test_split(tfidf_df, 
                                                    tfidf_df[estratifica], 
                                                    test_size=0.999, 
                                                    random_state=42)

# tfidf_df será test_size=1.0 -> corresponde a 100% do conjunto de dados
# Essa etapa foi criada para manter proporcional a amostragem e, assim, evitar o desbalanceamento.
# O desbalanceamento da amostra começa a ocorrer na medida que o número de dígitos do CEP aumenta,
# pois também crescerá o número de classes/cep a serem classificados e, consequentemente,
# a granularidade das classes aumentará.

tfidf_df.label = tfidf_df.label

del X_train, y_train

In [18]:
tfidf_df.head(1)

,log_nu,loc_nu,bai_nu_ini,bai_nu_fim,mun_nu,ufe_sg,estratifica,cep_2d,cep_3d,label,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
6392,141062,2704,3190,0,3104007,MG,381,38,381,3818,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [19]:
df_end.label.value_counts()

3940    549
3570    390
3550    322
3840    322
3755    313
       ... 
3012      2
3473      2
3631      1
3305      1
3225      1
Name: label, Length: 269, dtype: int64

Obs.: quero fazer estratificação por `estratifica` e manter `label` como target, mas o `train_test_split` usa `y` que é `label` para estratificar.  
- Buscar solução

In [20]:
# USAR ONEHOT ENCODER
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(tfidf_df['ufe_sg'])
tfidf_df['uf'] = label_encoder.transform(tfidf_df['ufe_sg'])

tfidf_df.uf.value_counts()

0    1924
Name: uf, dtype: int64

In [21]:
# Elimina a feature estratifica criada exclusivamente 
# quando for necessária uma amostragem estratificada proporcional por ufe_sg e cep
tfidf_df = tfidf_df.drop(columns=['estratifica'])
tfidf_df = tfidf_df.drop(columns=['ufe_sg'])

In [22]:
# cria o modelo ML baseado nos parâmetros
def create_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = XGBClassifier()
    model = model.fit(X_train, y_train)
    
    return model, X_test, y_test

In [23]:
# DELETA df_end e df para liberar RAM
del df_end, df

In [24]:
from sklearn.metrics import accuracy_score

# criar o modelo de classificação baseado no TFIDF
y_pred, X_test, y_test = create_model(tfidf_df, tfidf_df.label)

acuracia = accuracy_score(y_test, y_pred.predict(X_test))
print("Acurácia: %.2f%%" % (acuracia * 100.0))

Acurácia: 86.14%


- O primeiro teste de classificação considerando 4 dígitos do CEP como label (target).
    - Esse teste foi feito incluindo como feature o CEP com `2` e `3` dígitos
    - BOW = 2000
        - Resultado: Acurácia foi de 87.28%
    - BOW = 3000
        - Resultado: Acurácia foi de 88.24%
    - BOW = 3500
        - Resultado: Acurácia foi de 88.08%
    - BOW = 4000
        - Resultado: Acurácia foi de 88.39%
    - BOW = 4500
        - Resultado: Acurácia foi de 86.30%
    - BOW = 5000
        - Resultado: Acurácia foi de 86.14%
    - BOW = 6000
        - Resultado: Acurácia foi de 87.76%
    - BOW = 7000
        - Resultado: Acurácia foi de %